In [13]:

import os, shutil
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
import cv2
import shutil
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"




In [14]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

C:\Users\Mojtaba Jafari Tadi\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\client\session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [15]:
#get working directory
os.getcwd()

'C:\\Users\\Mojtaba Jafari Tadi\\Projects_AMK\\Radiology Project'

In [16]:
os.listdir()

['Binary_Classification_Split',
 'data_DICOM',
 'fatpad',
 'reformatted_binary',
 'reformatted_binary_unscaled',
 'reformatted_data',
 'reformatted_data_unscaled',
 'xray_model.h5']

In [17]:
#change working directory to where the datasets are
root_directory=os.chdir('C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project')

In [18]:
#Provide train, validation, and test to tensorflow-keras ImageDataGenerator
base_dir_train = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/train'
base_dir_validate = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/validation'
base_dir_test = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/test'

print(base_dir_train)

C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/train


In [19]:
#run tensorflow-keras ImageDataGenerator on the directory which contains train, validate and test
#train_datagen = ImageDataGenerator(
 #   rescale=1./255, # new parameter (maps pixel values to floats)
 #   rotation_range=40,
 #   width_shift_range=0.2,
  ##  height_shift_range=0.2,
  #  shear_range=0.2,
  #  zoom_range=0.25,
  #  horizontal_flip=True,
  #  fill_mode='nearest') # Note: I added this, too

train_datagen = ImageDataGenerator(samplewise_center=False, 
                              samplewise_std_normalization=False, 
                              horizontal_flip = True, 
                              vertical_flip = False, 
                              height_shift_range= 0.15, 
                              width_shift_range=0.15, 
                              rotation_range=5, 
                              shear_range = 0.1,
                              fill_mode = 'reflect',
                              zoom_range=0.15)

# The validation data should not be augmented (just rescaled)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        base_dir_train,
        color_mode='rgb',
        target_size=(224, 224), 
        batch_size=16, 
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        base_dir_validate, 
        color_mode='rgb',
        target_size=(224, 224), 
        batch_size=16, 
        class_mode='binary')

Found 3519 images belonging to 2 classes.
Found 1005 images belonging to 2 classes.


In [20]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


In [21]:
train_generator.image_shape


(224, 224, 3)

In [25]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [26]:
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[224, 224, 3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])


In [27]:

hist = model.fit(train_generator,
      epochs=50,
      validation_data=validation_generator,
      verbose=1)

plot_hist(hist)


Epoch 1/50
220/220 [==============================] - 76s 343ms/step - loss: 0.8148 - accuracy: 0.5359 - val_loss: 0.7315 - val_accuracy: 0.4866
Epoch 2/50
220/220 [==============================] - 73s 330ms/step - loss: 0.6976 - accuracy: 0.5658 - val_loss: 0.7059 - val_accuracy: 0.4866
Epoch 3/50
220/220 [==============================] - 73s 330ms/step - loss: 0.6913 - accuracy: 0.5726 - val_loss: 0.7455 - val_accuracy: 0.4866
Epoch 4/50
220/220 [==============================] - 73s 331ms/step - loss: 0.6808 - accuracy: 0.5806 - val_loss: 0.7359 - val_accuracy: 0.5134
Epoch 5/50
220/220 [==============================] - 73s 331ms/step - loss: 0.6727 - accuracy: 0.5973 - val_loss: 0.8733 - val_accuracy: 0.4866
Epoch 6/50
220/220 [==============================] - 73s 330ms/step - loss: 0.6745 - accuracy: 0.5956 - val_loss: 0.7269 - val_accuracy: 0.5134
Epoch 7/50
220/220 [==============================] - 73s 333ms/step - loss: 0.6686 - accuracy: 0.5962 - val_loss: 0.6940 - val_ac

KeyboardInterrupt: 

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=20)
test_acc